In [115]:
import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor


TODO:  
- aufsummieren der idpol-duplikate  anstelle dropping  
- Zielvariable ist claimamount / exposure  
- EDA selbst plotten  
- model fitting mit sample_weight=exposure

# Ziel dieses notebooks

Die gegebenen Datasets stellen viele Fragen. Dieses Notebook hat als Ziel, sehr schnell zu einem Modell zu kommen. Womöglich mit deutlichen Ungenauigkeiten.

# Importiere Datasets

In [116]:
df_sev = pd.read_csv('data/df_sev.csv', index_col=0)
df_freq = pd.read_csv("data/df_freq.csv", index_col=0)

In [117]:
display(df_sev.head(), df_freq.head())

,idpol,claimamount
0,1552.0,995.20
1,1010996.0,1128.12
2,4024277.0,1851.11
3,4007252.0,1204.00
4,4046424.0,1204.00


,idpol,claimnb,exposure,area,vehpower,vehage,drivage,bonusmalus,vehbrand,vehgas,density,region
0,1.0,1.0,0.10,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82'
1,3.0,1.0,0.77,'D',5.0,0.0,55.0,50.0,'B12',Regular,1217.0,'R82'
2,5.0,1.0,0.75,'B',6.0,2.0,52.0,50.0,'B12',Diesel,54.0,'R22'
3,10.0,1.0,0.09,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72'
4,11.0,1.0,0.84,'B',7.0,0.0,46.0,50.0,'B12',Diesel,76.0,'R72'


In [118]:
df_freq['claimnb'].unique()

array([ 1.,  2.,  4.,  3., 11.,  0.,  5.,  6.,  8., 16.,  9.])

# Datentypen

## df_sev

df_sev hat zwei Spalten. 

- Einmal die Spalte der Policenummer ('**idpol**') und dann 
- die Spalte der fälligen Aufwendungen für Versicherungsfälle ('**claimamount**').

In [119]:
df_sev.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26639 entries, 0 to 26638
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   idpol        26639 non-null  float64
 1   claimamount  26639 non-null  float64
dtypes: float64(2)
memory usage: 624.4 KB


### Gemischte Datentypen

In [120]:
for column in df_sev.columns:
    mixed_types = df_sev[column].apply(type).unique()
    
    if len(mixed_types) > 1:
        print(f"Spalte '{column}' hat gemischte Datentypen: {mixed_types}")
    else:
        print(f"Spalte '{column}' hat den Datentyp: {mixed_types[0]}")

Spalte 'idpol' hat den Datentyp: <class 'float'>
Spalte 'claimamount' hat den Datentyp: <class 'float'>


Beide Spalten liegen derzeit als Float 64 vor. 

- **idpol** ist *float64*, wird in *integer* umgewandelt, da categorical keinen Nutzen bringt.
- **claimamount** bleibt *float64*, um Genauigkeit zu behalten - auf Kosten der Rechner-Performance. 

#### Anpassung idpol

In [121]:
df_sev['idpol'] = df_sev['idpol'].astype('int32')

In [122]:
df_sev.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26639 entries, 0 to 26638
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   idpol        26639 non-null  int32  
 1   claimamount  26639 non-null  float64
dtypes: float64(1), int32(1)
memory usage: 520.3 KB


## df_freq

df_freq hat 12 Spalten.


In [123]:
df_freq.info()

<class 'pandas.core.frame.DataFrame'>
Index: 678013 entries, 0 to 678012
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   idpol       678013 non-null  float64
 1   claimnb     678013 non-null  float64
 2   exposure    678013 non-null  float64
 3   area        678013 non-null  object 
 4   vehpower    678013 non-null  float64
 5   vehage      678013 non-null  float64
 6   drivage     678013 non-null  float64
 7   bonusmalus  678013 non-null  float64
 8   vehbrand    678013 non-null  object 
 9   vehgas      678013 non-null  object 
 10  density     678013 non-null  float64
 11  region      678013 non-null  object 
dtypes: float64(8), object(4)
memory usage: 67.2+ MB


#### Gemischte Datentypen

Wir prüfen auf gemischte Datentypen.

In [124]:
for column in df_freq.columns:
    mixed_types = df_freq[column].apply(type).unique()
    
    if len(mixed_types) > 1:
        print(f"Spalte '{column}' hat gemischte Datentypen: {mixed_types}")
    else:
        print(f"Spalte '{column}' hat den Datentyp: {mixed_types[0]}")

Spalte 'idpol' hat den Datentyp: <class 'float'>
Spalte 'claimnb' hat den Datentyp: <class 'float'>
Spalte 'exposure' hat den Datentyp: <class 'float'>
Spalte 'area' hat den Datentyp: <class 'str'>
Spalte 'vehpower' hat den Datentyp: <class 'float'>
Spalte 'vehage' hat den Datentyp: <class 'float'>
Spalte 'drivage' hat den Datentyp: <class 'float'>
Spalte 'bonusmalus' hat den Datentyp: <class 'float'>
Spalte 'vehbrand' hat den Datentyp: <class 'str'>
Spalte 'vehgas' hat den Datentyp: <class 'str'>
Spalte 'density' hat den Datentyp: <class 'float'>
Spalte 'region' hat den Datentyp: <class 'str'>


Es liegen keine gemischten Datentypen vor.

In [125]:
df_freq['vehpower']

0         5.0
1         5.0
2         6.0
3         7.0
4         7.0
         ... 
678008    4.0
678009    4.0
678010    6.0
678011    4.0
678012    7.0
Name: vehpower, Length: 678013, dtype: float64

- **idpol** ist *float* und soll *string* sein [Skalenniveau: nominal].
- **claimnb** ist *float* und soll *integer* sein [Skalenniveau: verhältnis, diskret].
- **exposure** ist *float* und soll *integer* sein. [Skalenniveau: verhältnis, diskret]
- **area** ist *string* und soll *string* bleiben. [Skalenniveau: nominal - Strings sind zu bereinigen]
- **vehpower** ist *float*, sollte *categorical* werden [Skalenniveau: ordinal]

### Anpassung der Datentypen

# Duplikate

## df_sev

In [126]:
df_sev.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26639 entries, 0 to 26638
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   idpol        26639 non-null  int32  
 1   claimamount  26639 non-null  float64
dtypes: float64(1), int32(1)
memory usage: 520.3 KB


Hat df_sev Duplikate?

In [127]:
num_duplicates = df_sev.duplicated().sum()
print(f"Anzahl der Duplikate in df_sev: {num_duplicates}")

Anzahl der Duplikate in df_sev: 255


In diesem notebook entfernen wir einfach die Duplikate. Als Duplikat gilt, wenn die gesamte Datenreihe mehr als einmal vorkommt.

In [128]:
df_sev.drop_duplicates(keep=False, inplace=True)

In [129]:
num_duplicates = df_sev.duplicated().sum()
print(f"Anzahl der Duplikate in df_sev: {num_duplicates}")

Anzahl der Duplikate in df_sev: 0


## df_freq

In [130]:
df_freq.info()

<class 'pandas.core.frame.DataFrame'>
Index: 678013 entries, 0 to 678012
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   idpol       678013 non-null  float64
 1   claimnb     678013 non-null  float64
 2   exposure    678013 non-null  float64
 3   area        678013 non-null  object 
 4   vehpower    678013 non-null  float64
 5   vehage      678013 non-null  float64
 6   drivage     678013 non-null  float64
 7   bonusmalus  678013 non-null  float64
 8   vehbrand    678013 non-null  object 
 9   vehgas      678013 non-null  object 
 10  density     678013 non-null  float64
 11  region      678013 non-null  object 
dtypes: float64(8), object(4)
memory usage: 67.2+ MB


Hat df_freq Duplikate?

In [131]:
num_duplicates = df_freq.duplicated().sum()
print(f"Anzahl der Duplikate in df_freq: {num_duplicates}")

Anzahl der Duplikate in df_freq: 0


## Zusammenführung der Datasets

Da wir in diesem Notebook sehr zügig zu einem Modell kommen wollen, interessieren uns nur die Versicherungspolicen, denen ein Schadensaufwand zugeordnet werden konnte. Und es interessiert uns nur ein Schadensaufwand, dem Eigenschaften der Versicherungsfälle zugeordnet werden können. df_sev ist also im Lead - da claimamount die Zielvariable ist - und wir nutzen hier einen inner join.

In [132]:
df = df_sev.merge(df_freq, on='idpol', how="right")

In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679254 entries, 0 to 679253
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   idpol        679254 non-null  float64
 1   claimamount  25980 non-null   float64
 2   claimnb      679254 non-null  float64
 3   exposure     679254 non-null  float64
 4   area         679254 non-null  object 
 5   vehpower     679254 non-null  float64
 6   vehage       679254 non-null  float64
 7   drivage      679254 non-null  float64
 8   bonusmalus   679254 non-null  float64
 9   vehbrand     679254 non-null  object 
 10  vehgas       679254 non-null  object 
 11  density      679254 non-null  float64
 12  region       679254 non-null  object 
dtypes: float64(9), object(4)
memory usage: 67.4+ MB


In [134]:
df['claimamount'].fillna(0, inplace=True)

# Schnelle EDA 

Wir nutzen ydata profiling um den Dataframe zu analysieren.

In [135]:
#profile = ProfileReport(df, title="Profiling Report")
#profile.to_widgets()

## Multikollinearität - Korrelationen der unabhängigen Variablen

**drivage** is highly overall correlated with **bonusmalus**  
**bonusmalus** is highly overall correlated with **drivage**

Da **bonusmalus** als abhängig von **drivage** eingeschätzt wird (Kausalität), wird **bonusmalus entfernt**.

In [136]:
df.drop('bonusmalus', axis=1, inplace=True)

**density** is highly overall correlated with **area**  
**area** is highly overall correlated with **density**

In [137]:
df['density'].unique()

array([1217.,   54.,   76., ..., 1036., 1013.,  908.])

In [138]:
df['area'].unique()

array(["'D'", "'B'", "'E'", "'C'", "'F'", "'A'"], dtype=object)

Da der area-code (**area**) als abhängig von **density** eingeschätzt wird (Kausalität), wird der **area** entfernt.

In [139]:
df.drop('area', axis=1, inplace=True)

Es bleiben also folgende Features übrig:

In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679254 entries, 0 to 679253
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   idpol        679254 non-null  float64
 1   claimamount  679254 non-null  float64
 2   claimnb      679254 non-null  float64
 3   exposure     679254 non-null  float64
 4   vehpower     679254 non-null  float64
 5   vehage       679254 non-null  float64
 6   drivage      679254 non-null  float64
 7   vehbrand     679254 non-null  object 
 8   vehgas       679254 non-null  object 
 9   density      679254 non-null  float64
 10  region       679254 non-null  object 
dtypes: float64(8), object(3)
memory usage: 57.0+ MB


## Schnelle EDA (2)

In [141]:
#profile = ProfileReport(df, title="Profiling Report")
#profile.to_widgets()

**claimamount** ist rechtsschief. Wir ignorieren dies zunächst.

**vehage** hat 1173 Nullwerte. Das ist nicht schlimm, da es Autos gibt die jünger als ein Jahr sind.

## Letzte Datenbereinigung vor dem Splitting

Da die Nummer der Versicherungspolice offensichtlich keinen Einfluss auf den Schadensaufwand haben wird, lassen wir diese als Feature fallen.

In [142]:
df.drop('idpol', axis=1, inplace=True)

## Splitting

In [143]:
df_copy = df.copy()

In [144]:
y = df_copy['claimamount']
X = df_copy.drop('claimamount', axis=1)

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [146]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [147]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(543403, 9) (543403,)
(135851, 9) (135851,)


# Modellwahl und Vorbereitungen

Ich entscheide mich für **XGboost** als Gradient Boosting Model. Die lineare Regression ist nicht interessant, da ich nicht von linearen Zusammenhängen ausgehe. XGboost ist auch skalierungsunempflindlich, was helfen könnte wegen der schiefe der zielvariablen. Außerdem kann ich es über die GPU laufen lassen.

## Encoding der categorical data

In [148]:
X_train.isna().sum()

claimnb     0
exposure    0
vehpower    0
vehage      0
drivage     0
vehbrand    0
vehgas      0
density     0
region      0
dtype: int64

**preprocessing X_train**

In [149]:
one_hot = OneHotEncoder() #Objektinstanz Encoder

categorical_columns = ['vehbrand','vehgas','region'] #Definition Categorical Variables

encoded_data = one_hot.fit_transform(X_train[categorical_columns]).toarray() # Fitting und Encoding von allen cat. Variablen und Zuordnug zu encoded_df
encoded_df = pd.DataFrame(encoded_data, columns=one_hot.get_feature_names_out(categorical_columns))

train_cat_df = X_train[categorical_columns]
train_cat_df.isna().sum()


vehbrand    0
vehgas      0
region      0
dtype: int64

In [150]:
X_train = pd.concat([X_train, encoded_df], axis=1).drop(categorical_columns,axis=1)

In [151]:
X_train.isna().sum()

claimnb           0
exposure          0
vehpower          0
vehage            0
drivage           0
density           0
vehbrand_'B1'     0
vehbrand_'B10'    0
vehbrand_'B11'    0
vehbrand_'B12'    0
vehbrand_'B13'    0
vehbrand_'B14'    0
vehbrand_'B2'     0
vehbrand_'B3'     0
vehbrand_'B4'     0
vehbrand_'B5'     0
vehbrand_'B6'     0
vehgas_Diesel     0
vehgas_Regular    0
region_'R11'      0
region_'R21'      0
region_'R22'      0
region_'R23'      0
region_'R24'      0
region_'R25'      0
region_'R26'      0
region_'R31'      0
region_'R41'      0
region_'R42'      0
region_'R43'      0
region_'R52'      0
region_'R53'      0
region_'R54'      0
region_'R72'      0
region_'R73'      0
region_'R74'      0
region_'R82'      0
region_'R83'      0
region_'R91'      0
region_'R93'      0
region_'R94'      0
dtype: int64

**preprocessing X_test**

In [152]:
encoded_test_data = one_hot.transform(X_test[categorical_columns]).toarray()
encoded_test_df = pd.DataFrame(encoded_test_data, columns=one_hot.get_feature_names_out(categorical_columns))
test_cat_df = X_test[categorical_columns]
X_test = pd.concat([X_test, encoded_test_df], axis=1).drop(categorical_columns,axis=1)

**Create Model instance**

In [153]:
params = {
    'tree_method': 'gpu_hist',
    'n_estimators' : 1000,
    "learning_rate": 0.00001,
    "max_depth" : 5,
    "subsample" : 0.8,
    "colsample_bytree" : 0.8,
    "gamma" : 0,
    "min_child_weight" : 1,
    "objective" : 'reg:tweedie',
    "eval_metric" : "rmse",
}

bst = XGBRegressor(**params)

In [157]:
bst.fit(X_train,y_train, sample_weight=exposure)

TypeError: ('Unsupported type for weight', "<class 'str'>")

In [155]:
y_pred = bst.predict(X_test)

/home/bjzim/repos/mtpl2/.venv/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [12:13:49] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [156]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np

print(np.sqrt(mean_squared_error(y_test,y_pred)))
r2_score(y_test, y_pred)

1806.7980910594765


-0.0013531069429639775